In [1]:
import transformers
from transformers import CLIPConfig, CLIPModel, CLIPProcessor, CLIPImageProcessor, CLIPTokenizerFast
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler
import numpy as np
import random
import math
import scipy.io as sio
import nibabel as nib
from pathlib import Path
from gensim.models import Word2Vec
import re

In [64]:
mat_contents = sio.loadmat('subject_1.mat')
words = mat_contents["words"][0]
list_words = []
run = []
length = 4
context = 2
scan = []

def get_next_four(start_index, word_list):
    next_four = []
    end_index = 0
    a = 0
    end_of_run = False
    for x in range(4):
        if start_index+x < len(word_list):
            if word_list[start_index+x][2][0][0] <= 0.5:
                word = word_list[start_index+x][0][0][0][0]
                while('"' in word):
                    word = word.replace('"','')
                next_four.append(word)
            else:
                a = x
                end_of_run = True
                break
    if a == 0:
        a = 3
    end_index = start_index + a + 1
    return [" ".join(next_four),end_index,end_of_run]
                
index = 0           
while(index < len(words)):
    scan = []
    b = index
    end_of_run = False
    for y in range(context):
        if not end_of_run:
            a, b, end_of_run = get_next_four(b,words)
            scan.append(a)

    if not end_of_run:
        index += length
    else:
        index = b
    #print(scan)
    print(scan)
    list_words.append(" ".join(scan))

    """ 
    #print(x[2][0][0])
    if words[x][2][0][0] < 1:
        word = words[x][0][0][0][0]
        print(word)
        while('"' in word):
            word = word.replace('"','')
        scan.append(word)
        if len(scan) == length:
            run.append(" ".join(scan))
            scan = []
    else:
        if len(scan) != 0:
            run.append(" ".join(scan))
            scan = []
        list_words.append(run)
        run = []
        scan = []"""
        
print(list_words)
        

['Harry had never believed', 'he would meet a']
['he would meet a', 'boy he hated more']
['boy he hated more', 'than Dudley, but that']
['than Dudley, but that', 'was before he met']
['was before he met', 'Draco Malfoy. Still, first-year']
['Draco Malfoy. Still, first-year', 'Gryffindors only had Potions']
['Gryffindors only had Potions', 'with the Slytherins, so']
['with the Slytherins, so', "they didn't have to"]
["they didn't have to", 'put up with Malfoy']
['put up with Malfoy', 'much. Or at least,']
['much. Or at least,', "they didn't until they"]
["they didn't until they", 'spotted a notice pinned']
['spotted a notice pinned', 'up in the Gryffindor']
['up in the Gryffindor', 'common room that made']
['common room that made', 'them all groan. Flying']
['them all groan. Flying', 'lessons would be starting']
['lessons would be starting', 'on Thursday -- and']
['on Thursday -- and', 'Gryffindor and Slytherin would']
['Gryffindor and Slytherin would', 'be learning together. +']
['be l

In [62]:
print(len(list_words))
print(list_words)

1294
['Harry had never believed he would meet a', 'he would meet a boy he hated more', 'boy he hated more than Dudley, but that', 'than Dudley, but that was before he met', 'was before he met Draco Malfoy. Still, first-year', 'Draco Malfoy. Still, first-year Gryffindors only had Potions', 'Gryffindors only had Potions with the Slytherins, so', "with the Slytherins, so they didn't have to", "they didn't have to put up with Malfoy", 'put up with Malfoy much. Or at least,', "much. Or at least, they didn't until they", "they didn't until they spotted a notice pinned", 'spotted a notice pinned up in the Gryffindor', 'up in the Gryffindor common room that made', 'common room that made them all groan. Flying', 'them all groan. Flying lessons would be starting', 'lessons would be starting on Thursday -- and', 'on Thursday -- and Gryffindor and Slytherin would', 'Gryffindor and Slytherin would be learning together. +', 'be learning together. + Typical, said Harry darkly.', 'Typical, said Harry 

In [ ]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

inputs = processor(text=list_words[0], images=image, return_tensors="pt", padding=True)
outputs = model(**inputs)
# text_embeds is in the same space as the given image
text_embeds = outputs['text_embeds']

# pooler_output is just the text embedding 
# This is what we want to use I think 
pooler_output = outputs['text_model_output']['pooler_output']

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [ ]:
#print(pooler_output)
print(text_embeds)